$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
$$

# Part 1: Mini-Project
<a id=part3></a>

In this part you'll implement a small comparative-analysis project, heavily based on the materials from the tutorials and homework.

### Guidelines

- You should implement the code which displays your results in this notebook, and add any additional code files for your implementation in the `project/` directory. You can import these files here, as we do for the homeworks.
- Running this notebook should not perform any training - load your results from some output files and display them here. The notebook must be runnable from start to end without errors.
- You must include a detailed write-up (in the notebook) of what you implemented and how. 
- Explain the structure of your code and how to run it to reproduce your results.
- Explicitly state any external code you used, including built-in pytorch models and code from the course tutorials/homework.
- Analyze your numerical results, explaining **why** you got these results (not just specifying the results).
- Where relevant, place all results in a table or display them using a graph.
- Before submitting, make sure all files which are required to run this notebook are included in the generated submission zip.
- Try to keep the submission file size under 10MB. Do not include model checkpoint files, dataset files, or any other non-essentials files. Instead include your results as images/text files/pickles/etc, and load them for display in this notebook. 

## Object detection on TACO dataset

TACO is a growing image dataset of waste in the wild. It contains images of litter taken under diverse environments: woods, roads and beaches.

<center><img src="imgs/taco.png" /></center>


you can read more about the dataset here: https://github.com/pedropro/TACO

and can explore the data distribution and how to load it from here: https://github.com/pedropro/TACO/blob/master/demo.ipynb


The stable version of the dataset that contain 1500 images and 4787 annotations exist in `datasets/TACO-master`
You do not need to download the dataset.


### Project goals:

* You need to perform Object Detection task, over 7 of the dataset.
* The annotation for object detection can be downloaded from here: https://github.com/wimlds-trojmiasto/detect-waste/tree/main/annotations.
* The data and annotation format is like the COCOAPI: https://github.com/cocodataset/cocoapi (you can find a notebook of how to perform evalutation using it here: https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb)
(you need to install it..)
* if you need a beginner guild for OD in COCOAPI, you can read and watch this link: https://www.neuralception.com/cocodatasetapi/ 

### What do i need to do?

* **Everything is in the game!** as long as your model does not require more then 8 GB of memory and you follow the Guidelines above.


### What does it mean?
* you can use data augmentation, rather take what's implemented in the directory or use external libraries such as https://albumentations.ai/ (notice that when you create your own augmentations you need to change the annotation as well)
* you can use more data if you find it useful (for examples, reviwew https://github.com/AgaMiko/waste-datasets-review)


### What model can i use?
* Whatever you want!
you can review good models for the coco-OD task as a referance:
SOTA: https://paperswithcode.com/sota/object-detection-on-coco
Real-Time: https://paperswithcode.com/sota/real-time-object-detection-on-coco
Or you can use older models like YOLO-V3 or Faster-RCNN
* As long as you have a reason (complexity, speed, preformence), you are golden.

### Tips for a good grade:
* start as simple as possible. dealing with APIs are not the easiest for the first time and i predict that this would be your main issue. only when you have a running model that learn, you can add learning tricks.
* use the visualization of a notebook, as we did over the course, check that your input actually fitting the model, the output is the desired size and so on.
* It is recommanded to change the images to a fixed size, like shown in here :https://github.com/pedropro/TACO/blob/master/detector/inspect_data.ipynb
* Please adress the architecture and your loss function/s in this notebook. if you decided to add some loss component like the Focal loss for instance, try to show the results before and after using it.
* Plot your losses in this notebook, any evaluation metric can be shown as a function of time and possibe to analize per class.

Good luck!

## Implementation

# Section 0: Introduction

In this mini-project, we are tasked with implementing an object detection model on the TACO dataset. Our project goals are as follows:
* Modify the given TACO dataset to have 7 categories.
* Choose a viable object detection model.
* Train the chosen model on the dataset.
* Improve the model results.


# Section 1: Dataset

In this project we are working with the TACO dataset. The dataset images were scraped from various online sources and humans annotated the images. TACO stands for Trash Annotations in Context and it is a dataset specifically designed for object detection and segmentation tasks related to identifying and classifying different types of waste in images. It has 28 super categories and 60 categories, with the following distribution [4]:

<img src="imgs/rawTacoDist.png" />

The data and annotation in the TACO dataset are structured similarly to the COCOAPI. The familiar structure and conventions make it convenient to leverage the COCOAPI's functionality, such as data loading, annotation parsing, and evaluation metrics. Inside our TACO dataset we have a folder for the images, each image has: id (img), width, height, file_name among other categories. We also have a separate folder for annotations, each annotation has: id (ann.), img_id, category_id, area, bbox (x,y,width,height), and is_crowd. In the dataset there are 1500 images, and 4784 annotations. We note that there are more annotations than images because an image can have multiple annotations. For example, the following image has 6 annotations. 

<img src="imgs/expAnn.png" />

We want to reduce the number of categories to 7. We want to do this for several reasons:
* Simplification and Focus: we believe that working with a smaller number of categories simplifies the task, making it more manageable and focused. This is important to us because we have access to limited computational resources, time, and data.
* Resource Efficiency: training an object detection model on a large number of categories can be resource intensive in terms of computational power and memory. Reducing the categories can make training more efficient and cost effective.
* Improved Generalization: an object detection model trained on a smaller set of categories may generalize better to new, unseen data because it focuses on learning more discriminative features for the chosen categories.

We decided to split up our original TACO categories like so [11]:

<img src="imgs/tacoSplit.jpeg" />

These new categories provide a balanced and comprehensive representation of different waste types and materials commonly encountered in waste management and recycling scenarios. They distinguish between recyclable and non-recyclable materials, as well as handle unclassified or ambiguous items. Moreover, this selection aligns with TACO’s objectives of providing a practical approach to waste classification and management [12].

Here is the distribution of the new TACO categories:
Note: ‘other’ has 2 annotations, and ‘bio’ has 8 annotations.

<img src="imgs/newTacoDist.png" />

As we can see, the distribution stays fairly similar before and after combining the categories from 60 to 7.

In section 2, we talk about why we decided to not resize our images to be the same size.

To reproduce these results, you can run the TACO_dataset_conv.ipynb notebook. This notebook loads in the original TACO dataset that is on the lambda server, runs a script to split up the categories, and creates a new dataset with these new categories. Credit is given in the notebook for each external code used. 
* The primary purpose of this code is to map categories from the TACO dataset to categories in the Detectwaste dataset. The 'taco_to_detectwaste' method is responsible for mapping a single TACO category to a Detectwaste category based on predefined mappings. It checks the input category against predefined lists and assigns the appropriate category. The 'taco_categories_to_detectwaste' method takes as input the path to a TACO dataset file and a destination path. It loads the TACO dataset, updates category information, and assigns new category IDs based on the Detectwaste categories. The function essentially converts the annotations in the TACO dataset to match the Detectwaste categories. Throughout the code, it maintains information about categories, supercategories, category IDs, and supercategory IDs.


# Section 2: Model Selection - Faster RCNN

In class we learned about two types of object detection models: two stage models (RCNN) and one stage models (yolo). Both have their advantages and disadvantages. One stage models are usually faster to train and faster at run time, but can also be less accurate than two stage models. One stage models, because of their fast real time performance, are great in scenarios like self-driving cars. But, in our scenario, we can afford our model to be slower at run time if that means that it will be more accurate. Another advantage that we came across for two stage models is that Faster R-CNN is far more compatible with the TACO (through the use of torchvision). The use of YOLO, on the other hand, does not use JSON datasets and would require more sophisticated techniques to overcome this obstacle [9]. All in all, Faster-RCNN was the model we chose for our object detection task. From the graph below, we can see that Faster RCNN models perform reasonably well on COCO, and because TACO is similar to COCO, we expect the faster RCNN model to perform ok on TACO.

<img src="imgs/coco_graph.png" />

Here is a very simplified pipeline of how the Faster-RCNN forward pass looks like [1]:
1. The input image passes through the CNN backbone to extract feature maps.
2. The RPN operates on the feature maps to propose potential RoIs along with confidence scores.
3. RoI pooling converts the RoIs into a consistent size.
4. The object classification head assigns class labels to each RoI.
5. The bounding box regression head refines the RoI coordinates.
6. NMS is applied to remove duplicate and low confidence detections.
7. The final object detections, including class labels and bounding boxes, are obtained.

<img src="imgs/faster_rcnn_architecture.png" />

Note about resizing images in our dataset:
We decided to not resize the images in our dataset to all be the same size, like suggested in the project outline. Our decision not to resize the images was driven by a desire to maintain the integrity of the dataset, preserve aspect ratios, and leverage the faster RCNN’s adaptability to varying object scales. While resizing images can be a useful preprocessing step in certain contexts, in our specific case, we reached the conclusion that our approach is acceptable and aligns with our objectives.

**Losses**:

* RPN Box Regression Loss - smooth L1 loss: This loss corresponds to the regression loss associated with the RPN. It measures how well the RPN predicts the bounding box offsets for the proposed regions set by the anchors. This loss guides the RPN in refining the region proposals.

* RPN Classifier / Objectness Loss - binary CE loss: This loss is associated with the binary classification task performed by the RPN. It evaluates how well the RPN distinguishes between regions that contain objects and those that do not. This loss encourages the RPN to generate high-quality region proposals.

* RoI Box Regression Loss - smooth L1 loss: The RoI box regression loss corresponds to the regression loss applied to the bounding box offsets for RoIs in the object detection head. This loss guides the network in refining the predicted bounding box coordinates for the RoIs.

* RoI Classifier Loss - softmax CE loss: This loss measures the accuracy of object classification within RoIs. It evaluates how well the model classifies RoIs into specific object categories. This loss is associated with the object detection head and ensures that the model correctly identifies object classes within proposed regions.

* Total Loss: This loss is a weighted sum of the above 4 losses. The weights are chosen empirically to balance the contributions of each loss component during training. Like how a loss should behave, minimizing this total loss during training leads to improved object detection performance.

**Evaluation Metrics**:
We will evaluate our model using the AP and AR metrics at different IoU thresholds and object area sizes. These are the IoU thresholds that we will use: IoU=0.50 (low overlap), IoU=0.75 (moderate overlap), and IoU=0.50:0.95 (a range of IoU values). The object sizes are as follows: small, medium, large, all sizes. We can think of AP as "Out of all the times our model says something is a positive case, how often is it actually correct?" and we can think of AR as "Out of all the actual positive cases, how many did your model manage to find?”. Overall, a higher AP and AR indicate better model performance. The following is an example of evaluation on the test set:

<img src="imgs/evalExp.png" />



# Section 3: Training our Model with No Data Augmentations

First we decided to learn how to train a faster RCNN model without any tricks, like it was recommended in the project instructions. Because of our relatively small dataset, we decided to finetune an already trained faster RCNN model, rather than train one completely from scratch.
We followed the following tutorial by torchvision on object detection finetuning [13].

To view the code, please check out: train_file_no_aug.py

Overview of the code:
1. Setting the Random Seed: A random seed is set to ensure reproducibility of results.
2. Dataset Loading: We load our annotations file (new annotations that we created).
3. Custom Dataset Class - TacoDataset: We define a custom pytorch dataset. The class handles tasks such as loading images, applying data transformations, and preparing target information (bbox, labels) for object detection.
4. Model Initialization - get_model func: A pre-trained Faster R-CNN model with a ResNet-50 backbone is initialized. The classifier head of the model is replaced with a new one to match the number of object classes in the our custom TACO dataset, of which there are 8 (7 foreground classes and 1 background class).
5. Data Augmentation - get_transform func: We did not use any data augmentations, it was just convenient to include this because we needed to transform an input image from our dataset to a format that our model can use. Notice how we set train=False in our dataset creation inside of the training loop.
6. Training Loop - main func: First we select what device to train on. We define our TacoDataset and split it up into 80 20 train test split. Then we define our dataloaders (train,test), which are only of a size of 2 because any higher will cause memory problems. We initialize our model, optimizer (SGD), and a learning rate scheduler. We then run our training loop for 30 epochs. Inside each loop we use the func train_one_epoch. It updates the model weights, computes losses, and provides various metrics. We log the metrics using tensorboard. Update our lr scheduler and save a checkpoint of our model. At the end of each epoch, we evaluate our model on our test set, and we want to emphasize that there is no data leakage because we just test on our test set, and do nothing with it. We don't use it to update our model, we don't even look at it until the very end of the 30 epochs.

Note: our graphs are from tensorboard.

Evaluation:
We ran the train_file_no_aug.py script using the bash script train_file_no_aug.sh. From this we were able to derive the following results.

<img src="imgs/training_loss_no_augs.jpeg" />

As we can see the model converges toward the end of the 30 epochs. This tells us that we chose the adequate hyperparams and that our model is learning.
Note: we chose our hyperparams by trial and error.

We can also graph the 4 parts of the total loss. We can see that all 4 converge by the end of the 30 epochs, but some converge earlier than others. We think that this is related to the complexity of the problem each loss is trying to optimize, for example box regression is far more complicated than objectness.

<img src="imgs/noAug4.png" />

Additionally, we were able to obtain the following graphs showing the average precision and average recall of the model when acting on the test set as a function of the epoch number. Note: each graph corresponds to specific IoU, area and maximum number of detection values.

<img src="imgs/valid_no_augs.png" />

Average Precision (AP):
* IoU=0.50:0.95 | area=all | maxDets=100: The average precision over all object sizes and IoU thresholds with a maximum of 100 detections is between 0.061 and 0.119. This metric provides an overall assessment of the model's performance.
* IoU=0.50 | area=all | maxDets=100: Similar to the previous metric but with a fixed IoU threshold of 0.50, it ranges from 0.107 to 0.186.
* IoU=0.75 | area=all | maxDets=100: This measures the model's precision at a higher IoU threshold of 0.75 and ranges from 0.067 to 0.136.

Average Recall (AR):
* IoU=0.50:0.95 | area=all | maxDets=1: This measures the model's recall with a single detection. The value ranges from 0.110 to 0.158.
* IoU=0.50:0.95 | area=all | maxDets=10: This metric represents the recall with up to 10 detections, and it ranges from 0.248 to 0.277.
* IoU=0.50:0.95 | area=all | maxDets=100: The recall with a maximum of 100 detections ranges from 0.272 to 0.315.

We see that in most cases, the average precision and average recall begin to converge to relatively stable values. The AP is higher for larger objects, indicating better detection for larger objects. This is a common trend in object detection, where larger objects are often easier to detect. Moreover, we obtained much greater values of Average Recall than Average Precision. But we notice that the AR initially increases with training but very quickly decreases very rapidly. We think that this is happening because our model is overfitting to the train data. As the model learns the training data better, it may become less generalizable to the test data, leading to lower recall on unseen examples. Although the model performs much worse on this TACO dataset than on the COCO dataset, this is to be expected as some transfer learning tasks perform better on the original dataset on which they were trained, and we didn’t use any “learning tricks”, we just wanted to get a model up and working.

# Section 4: Model Improvement Techniques

After training our model without any sort of data augmentations, we achieved subpar results, mainly because of the low precision and overfitting. According to [3], there are mainly 3 ways to improve the performance of our object detection model: change the inherent model architecture, increase the computing power, or train on more data. We cannot get any more computing power, so we did not try that option. First, we decided to focus our efforts on trying to get more data to train our model on. The new data we find should come from a similar distribution as the original dataset that we were given [4]. Secondly, we tried to change up our model’s architecture.

# Section 5: Training our Model with Data Augmentations

**Why use Data Augmentation?:**

The dataset that we are working with [4] has a total of 1500 images. We decided to split up the train and test dataset with a 80/20 split. Practically, this means that we train our model with only 1200 images. This is very small in comparison to the COCO dataset [6], in which there are over 300,000 annotated images. Because of this, we would like to increase the number of images that are available to our model to train on. One way to do this is to use data augmentations. The core idea of data augmentation is to improve the sufficiency and diversity of training data by generating a synthetic dataset. The augmented data can be regarded as being extracted from a distribution that is close to the real one [3]. 

**What Data Augmentations did we use?:**

First we’ll define the 5 data augmentations that we used:
1. RandomHorizontalFlip: Horizontally flips the image. The objects in our dataset can come in either their original format or in their flipped format, for example the object below can come in either horizontal direction:

<img src="imgs/random_horizontal_flip.png" />


2. RandomIoUCrop: Random cropping of the image, while keeping at least one object (bbox) in the new cropped image. This simulates scenarios where objects are partially occluded and vary in size within an image, resulting in a model that is more robust at handling complex and diverse scenes.
Example:

<img src="imgs/random_crop.png" />

3. RandomZoomOut: Performs random zoom out of an image. The idea is to increase the canvas size of the image, resulting in an empty space around the image. This emulates the objects as a part of a larger scene.
Example:

<img src="imgs/random_zoom_out.png" />

4. RandomPhotometricDistort: Applies a random photometric distortion to the image, i.e. modifications to the image’s color and intensity. This creates variations in lightning and appearance.
Example:

<img src="imgs/random_photometric.png" />

5. ScaleJitter: Randomly resizes images and their associated bounding boxes within a specified range. This introduces variability in object scales during training, which helps improve the model’s ability to detect and recognize objects of different sizes.
Example:

<img src="imgs/scale_jitter.png" />

Full Data Transformation Pipeline:
* Note: We applied this data transformation pipeline on the **training set only**, and not on the test set, as it is good practice not to tamper with the test set.
* Notice how each data augmentation is applied with a certain probability.

<img src="imgs/transform_code.png" />

Examples of full data augmentation pipeline:

Original Image 1:

<img src="imgs/original_image1.png" />

Augmented Image 1: horizontal scale was applied as well as zoom in and photometric distort. This new augmented image looks like it could have come from our dataset.

<img src="imgs/aug_image_1.png" />

Original Image 2:

<img src="imgs/original_image2.png" />

Augmented Image 2: zoom out, photometric distort and IoU crop was applied. This new image, in theory,  will help the model recognize a “bottle/metels_and_plastic” as a part of a larger scene.

<img src="imgs/aug_image_2.png" />

**Training:**

Our train file is called train_file_aug.py. 
It is very similar to train_file_no_aug.py but when we create our training dataset, we set train=True which activates the data transforms.
We used our train_file_aug.sh bash script to run our train loop for 30 epochs on the server.

Here are the results:

<img src="imgs/loss_augs.jpeg" />

We can see that the overall training loss gradually converges.

<img src="imgs/augLoss4.png" />

From our breakdown of our total loss into its 4 sub losses we found that all 4 sub losses also converged by epoch 30, although each one at a slightly different rate.

As before, we were able to obtain the following graphs showing the average precision and average recall of the model when acting on the test set as a function of the epoch number.

<img src="imgs/valid_both.png" />

In comparison to the original model, we see that the model that was trained on augmented data obtains very slightly worse values of precision than the original model. This is most likely due to randomness.

However, the augmented model also obtains far greater values of recall. This is because augmentations such as random cropping, flipping, and resizing can help expose the model to a wider variety of object scales, orientations, and poses. This increased diversity can make the model more robust and capable of detecting objects that might have been missed without these augmentations. As a result, the recall tends to improve since the model is better at finding objects under different conditions. This can also be explained by the fact that our original model was overfitting to the small train dataset that we were given. By using data augmentations, our original dataset is effectively much larger, causing the augmented model to not overfit.

Overall, we can say that due to the heavy increase in recall (with only a mild loss of precision), the use of data augmentations is beneficial to the model.



# Section 6: Training our Model with a Different Backbone

After getting back our results from the augmented model, we were happy that our recall improved and that our model was not overfitting the training data anymore, but we were still not happy with the precision. We think that this might be because of the inherent architecture of our model. We looked into changing 3 things about our model:
* Backbone: a CNN, inputs an image, and outputs features. In our original model, we used ResNet-50. Because the training took a couple days, we decided to go with a more light weight model: MobileNetV2. This model is a bit less accurate but it should train faster, which is important for us because we are quite close to the deadline. Also we just wanted to experiment with another CNN architecture. 
* Anchor Generator: responsible for generating a set of candidate anchor boxes of different sizes and aspect ratios at multiple positions across the feature maps. These anchor boxes serve as potential object proposals. In the default settings, torchvision uses predefined anchor sizes and aspect ratios suitable for common object detection tasks. Getting the anchor generator right will help our model pick up on small and medium sized objects better, something that is a weak point in our current model.
* RoI Pooler: crops and aligns regions of interest from the feature maps produced by the backbone. It then extracts fixed-size feature maps from the corresponding features of the backbone. This operation is crucial because it ensures that object features are consistently aligned, regardless of their positions or sizes within the proposals. The sampling_ratio controls the level of detail when performing this alignment.

In our code, which you can find in backbone_no_augs.py, we kept the same as train_file_no_aug.py but changed our get_model method. We first define our backbone to be a pretrained MobileNetV2 model from torchvision. We set our backbone’s out_channels parameter to be the same as MobileNetV2, which is 1280. We define an AnchorGenerator for the RPN, specifying the anchor sizes and aspect ratios that the RPN will use. We then configured our ROI Pooling operation to perform pooling on the feature map indexed by '0' (the MobileNetV2 backbone's output feature map). It resizes each region proposal to a fixed 2D output_size and performs bilinear interpolation to align the features properly.  We were not completely sure what parameters to put so we went with what was online [13]. Lastly, we defined our new Faster RCNN model.

We trained our new backbone model on data that we did not apply data augmentations to. We wanted to do this for 2 reasons:
* We wanted to see how changing the backbone will impact performance, and not have too many variables interacting with each other all at once.
* Training time might be too long, and we won't get back our results by the project deadline.

We proceeded to run the backbone_no_augs.py script using the bash script backbone_no_augs.sh on the lambda server.

<img src="imgs/backbone_total_loss.jpeg" />

As we can see from the total loss, the model converged toward the end of the 30 epoch training loop.

<img src="imgs/backboneLoss4.png" />

If we look at the breakdown of the total loss into its 4 sub losses, we also see this converging behavior for each loss, but they converge at different rates.

As before, we were able to obtain the following graphs showing the average precision and average recall of the model when acting on the test set as a function of the epoch number. From this we were able to derive the following results:

<img src="imgs/backbone_evaluation.png" />

The first thing we notice is that our AP did not improve, but it did stay fairly close with the previous 2 models. MobileNetV2 is less deep than ResNet50, therefore having less capacity to capture the intricate details of the data. Also, we wanted our precision on small and medium objects to increase, therefore changing to a lighter model was not a good design choice. On the other hand, we see that our AR improved to the level of our augmented model. This makes sense because MobileNetV2 is a model that is not as deep as ResNet50, therefore making it less capable to overfit the train data, like what ResNet50 was doing.


# Section 7: Closing Remarks - Further Improvements

Unfortunately the project deadline is too close to be able to experiment with more ways to improve our model. We are sure that given more time, we would be able to improve its precision. The following are a few ways that we thought about that might improve our model:
* Modify our backbone parameters to better suit the TACO dataset, as well as train with data augmentations. We can use anchor optimization techniques to generate anchor boxes tailored to your dataset. Tools like k-means clustering can help find optimal anchor box sizes and ratios.
* Balance sampling: In the dataset section we noticed that some categories are far more represented than others. We might want to oversample these under-represented classes or use techniques like online hard example mining to focus on difficult examples.
* Loss Function Modification: We can try to experiment with different loss functions. It was recommended for us in the project instructions to use focal loss to handle class imbalance and improve localization accuracy.


# Section 8: References

1. Gad, A.F. (2021) Faster R-CNN explained for Object Detection Tasks, Paperspace Blog. Available at: https://blog.paperspace.com/faster-r-cnn-explained-object-detection/ (Accessed: 22 August 2023). 
2. Papers with code - coco test-dev benchmark (object detection) (no date a) The latest in Machine Learning. Available at: https://paperswithcode.com/sota/object-detection-on-coco (Accessed: 22 August 2023). 
3. Yang, S. et al. (2022) Image Data Augmentation for Deep Learning: A Survey, arXiv.org. Available at: https://arxiv.org/abs/2204.08610 (Accessed: 27 August 2023). 
4. Pedropro (no date) Pedropro/taco: 🌮 trash annotations in context dataset toolkit, GitHub. Available at: https://github.com/pedropro/TACO (Accessed: 27 August 2023). 
5. Ren, S. et al. (2016) Faster R-CNN: Towards real-time object detection with region proposal networks, arXiv.org. Available at: https://arxiv.org/abs/1506.01497 (Accessed: 27 August 2023). 
6. Lin, T.-Y. et al. (2015) Microsoft Coco: Common Objects in Context, arXiv.org. Available at: https://arxiv.org/abs/1405.0312 (Accessed: 27 August 2023). 
7. Gandhi, R. (2018) R-CNN, fast R-CNN, Faster R-CNN, YOLO - object detection algorithms, Medium. Available at: https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e (Accessed: 27 August 2023). 
8. Shivaprasad, P. (2020) A comprehensive guide to object detection using YOLO framework - part I, Medium. Available at: https://towardsdatascience.com/object-detection-part1-4dbe5147ad0a (Accessed: 27 August 2023). 
9. Rehman, A. (2023) Converting A custom dataset from coco format to Yolo Format, Medium. Available at: https://medium.com/red-buffer/converting-a-custom-dataset-from-coco-format-to-yolo-format-6d98a4fd43fc (Accessed: 29 August 2023). 
10. Ahmed, I. (2023) Why your mobilenetv2 model performs better than resnet50, Medium. Available at: https://medium.com/@imtiaz.ahmed2206/why-your-mobilenetv2-model-performs-better-than-resnet50-2a9998fda4c7#:~:text=MobileNetV2%20is%20much%20easier%20to,validation%20set%20with%20little%20tweaks. (Accessed: 02 September 2023). 
11. GitHub. Available at: https://github.com/wimlds-trojmiasto/detect-waste/blob/main/utils/dataset_converter.py (Accessed: 06 September 2023). 
12. Trash annotations in context (no date) tacodataset.org. Available at: http://tacodataset.org/ (Accessed: 06 September 2023). 
13. TorchVision Object Detection Finetuning Tutorial — PyTorch Tutorials 2.0.1+cu117 documentation. (n.d.). https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html (Accessed: 06 September 2023). 